# <img src="http://gi-mosm.dem.isel.pt/compdrill/img/logo_isel.png" style="width:300px; float: right; margin: 0 40px 40px 40px;"></img>

### Instituto Superior de Engenharia de Lisboa
### Licenciatura Em Informática e Multimética
# Aprendizagem Automática

## TRABALHO PRÁTICO 

Trabalho realizado por:
    
* Gonçalo Almeida - 43746
* Luís Fonseca    - 45125

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
## Indice
-> [Introdução](#introducao)<br/>
-> [Classificadores Escolhidos](#classificadores)<br/>
-> [Vetorização do Texto](#vetores)<br/>
-> [Classificação Binária](#binario)<br/>
-> [Classificação Multiclass](#multiclass)<br/>
-> [Regressão Linear](#linear)<br/>
-> [Dimensão do Dicionário](#dimensao)<br/>
-> [Conclusões](#conclusao)<br/>
-> [Bibliografia](#bibliografia)<br/>

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
## Introdução<a name="introducao"/>

O trabalho prático de Aprendizagem Automática consiste na aplicação de diferentes classificadores, e escolher, qual obtém uma melhor classificação. Para testarmos os diferentes classificadores, usamos o ficheiro fornecido pelo docente responsável da disciplina. Esse ficheiro, de nome "IMDBcríticas" consiste em 40000 diferentes críticas, irá ser usado para testar os classificadores aqui escolhidos. É de notar que, visto que estamos a trabalhar com um grande conjunto de dados, muitos dos classificadores aqui usados, apresentam um tempo de processamento excessivo, portanto apenas iremos testar com uma pequena parte dos dados, assim como, usar a maior parte dos parâmetros que foram testados.

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
## Classificadores Escolhidos<a name="classificadores"/>

Visto que foi a realização do projeto foi em grupo, foram escolhidos três classificadores, e mais outros dois tópicos opcionais. Os classificadores aqui usados foram os seguintes:

* RegressãoLogística(ou LogisticRegression): usado para modelar a probabilidade de uma determinada classe ou evento existir, como aprovação/reprovação, vitória/derrota, vivo/morto. 
* Máquina de Suporte Vetorial(Suport Vetorial Machine or SVM): é um modelo de aprendizagem supervisionada com algoritmos de aprendizagem associados que analisam dados para classificação e análise de regressão.
* KNN: usado para classificação e regressão. Em ambos os casos, a entrada consiste nos k exemplos de treinamento mais próximos no conjunto de dados. A saída depende se k-NN é usado para classificação ou regressão. Na regressão k-NN, a saída é o valor da propriedade do objeto.

E os tópicos opcionais escolhidos foram os seguintes:

* Regressão Linear: abordagem linear para modelar a relação entre uma resposta escalar e uma ou mais variáveis explicativas (também conhecidas como variáveis dependentes e independentes).


* Dimensão do Vocabulário: investigar a influência do tamanho do dicionário(dimensão dos dados) no desempenho de um discriminante logístico no problema de classificação binária.



![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
## Vetorização do Texto <a name="vetores"/>
Como primeiro passo, foi necessário dividir os dados de texto, em treino e de teste, para se possam realizar uma avaliação correta da quantidade de acertos de cada classificador. Como primeiro passo para a realização da vetorização, recorremos às bibliotecas da linguagem python, uma delas de nomede nome *`sklearn`* contendo os métodos necessários que permite fazer a respetiva conversão.

Recorremos também à biblioteca *`nltk`* para aplicar *`stemming`*, durante a realização, testamos todos os tipos de stemmers, sendo que foi usado o *`PorterStemmer`* com mais frequência.

Como último método, usamos o método *`train_test_split`* que permite efetuar a divisão dos dados em treino e teste.

In [1]:
import numpy as np 
import pickle
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer
from sklearn.model_selection import train_test_split

Usamos também a biblioteca *`pickle`* que permite abrir ficheiros dentro do formato pickle, e guardar o mesmo ficheiro, aplicando limpeza e o stemming usado. A seguir podemos ver a biblioteca *`pickle`* a ser usada, para abrir o ficheiro que foi fornecido.

In [2]:
D=pickle.load(open('imdbCriticas.p','rb'))

Com o ficheiro aberto, procedemos à limpeza do ficheiro. Para isso foi criado o método *`clear_test(original_data)`* que ao ser passado o ficheiro como argumento da função, permite eliminar mudanças de linhas dentro do ficheiro, e substituir tudo que não seja caracteres alfabéticos.

In [3]:
"""Método que recebe um documento, fazendo a sua limpeza"""
def clear_text(original_data):
    
    print("Start text clearing")
    data = [doc.replace('<br />',' ') for doc in original_data] # substituir as mudanças de linha
    data2 = [re.sub(r'[^a-zA-Z]+', ' ',doc)for doc in data] # tirar tudo que não é caracteres alfabeticos fora
    print("Text clearing finished")
    return data2

De seguida, criamos o método *`stemming(original_data,stemType="Porter", language = "english")`* que permite aplicar stemming, depois de ter sido feito a limpeza do ficheiro. Para este método, passamos os três tipos de stemers que existem.

In [4]:
"""Aplica os 3 tipos de stemming ao documento recebido"""
def stemming(original_data,stemType="Porter", language = "english"):
    
    stemmers = {"porter": PorterStemmer(),
            "snowball": SnowballStemmer("english"),
            "lancaster": LancasterStemmer()}
    stemmer = stemmers[stemType.lower()]
    print("Start stemming: ",stemType,"Stemmer\n")
    stemmed_data = [" ".join([stemmer.stem(word) for word in text.split()]) for text in original_data]
    print("Stemming finished.\n")
    return stemmed_data


Com o stemming aplicado, e a limpeza feita, criamos o método *`save_text(Data,Target.stemType)`* que permite guardar um novo ficheiro, depois das alterações feitos. Dentro deste ficheiro, guardamos os novos dados (*`Data`*), assim como o *`Target`*. No final usamos o método *`dump`* da biblioteca *`pickle`* para guardar o novo ficheiros, com os dados mais recentes.

In [5]:
"""Método que guarda o texto, depois de ter feito uma limpeza"""
def save_text(Data,Target,stemType):
    
    D = {'data' : Data,'target' : Target}
    print("Writing data.")
    pickle.dump(D, open('imdbFull' + str(stemType) + '.p', 'wb')) 
    print("Data written successfully")

Foi também criado o método *`save_data(data,name)`* que permite guardar os classificadores num ficheiro, após ter sido aplicado o método fit, utilizando os dados de treino, sobre os mesmos.

In [6]:
"""Método que permite guardar os novos dados"""
def save_data(data,name):
    
    D = {'data' : data}
    print("Writing data.")
    pickle.dump(D, open(str(name) + '.p', 'wb')) 
    print("Data written successfully")

Antes de passar para a divisão do texto para vetores, efetuamos um teste, comprovando o correto funcionamento dos métodos criados anteriormente, passando o tipo de stemming com sendo "Porter".

In [7]:
D=pickle.load(open('imdbCriticas.p','rb'))
Data=D['data']
Target=D['target']
stemType = "Porter"
clean_data = clear_text(Data)
stemmed_data = stemming(clean_data,stemType)
save_text(stemmed_data, Target, stemType)
#x_train, x_test, y1, y2 = train_test_split(Data, Target)
#x_train2, x_test2, voc2 = textoSplitTreinoTeste(Data[:50],Data[50:150], n_grams = (1,3))

Start text clearing
Text clearing finished
Start stemming:  Porter Stemmer

Stemming finished.

Writing data.
Data written successfully


Depois de ter sido guardado o novo ficheiro, procedemos à realização da divisão do ficheiro, em treino e teste. Para isso foi criado o método *`text_to_vetor(Xtrain,Xtest,ytrain, ytest, min_doc_freq = 5, language = 'english',n_grams = (1,3))`*, recebendo como argumentos os dados de treino e de teste, um min_doc_freq igual a 5, verificando apenas 50% dos documentos, e passando n-gramas, de (1,3).

Recorremos ao método *`TfidfVectorizer`* para aplicar esse conjunto de dados, numa matriz tfidf, passando os n-gramas e a quantidade mínima (*`min_df`*) 

No final, aplicamos esta transformação, tantos para os dados de treino, como para os dados de teste. No final, retornamos os dados de teste e de treino convertidos, assim como um vocabulário.

In [8]:
"""Método que converte o texto numa matriz(X) """
def text_to_vector(Xtrain,Xtest,ytrain, ytest, min_doc_freq = 5, language = 'english',n_grams = (1,3)):
    
    print("Converting text into vectors.")
    pattern = r'\b\w\w\w\w+\b' if n_grams == (1,1) else r'\b\w\w+\b'
    tfidf = TfidfVectorizer(min_df = min_doc_freq, token_pattern = pattern, ngram_range = n_grams, stop_words = language)
    tfidf.fit(Xtrain)
    save_data(tfidf,"tfidf_treinado")
    voc = tfidf.get_feature_names()
    
    #Projecção dos vectores
    x_train = tfidf.transform(Xtrain)
    x_test = tfidf.transform(Xtest)
    print("Finished conversion.")
    return x_train, x_test, ytrain, ytest, voc

O método acima descrito é apenas utilizado para efeitos de teste, sendo ainda necessário criar o método *`text2vector`*.

Este método recebe um dicionário composto por Data e Target. De modo a obter estes valores é evocado o método *`unpack`*, sendo, seguidamente, realizada a limpeza do texto e o stemming do texto limpo.

Por fim vai-se obter o tf-idf previamente treinado e, aplicando o mesmo, é realizada a vetorização destes dados, sendo, por fim, os mesmos guardados novamente num dicionário.


In [9]:
"""Método que permite correr a vetorização por texto"""
def text2vector(Docs):
    
    Data, Target = unpack(Docs)
    clean_data = clear_text(Data)
    stemmed_data = stemming(clean_data,"Porter")
    
    fName="tfidf_treinado.p"
    D = pickle.load(open(fName, 'rb'))
    tfidf = D['data']
    
    vector_data = tfidf.transform(stemmed_data)
    X = {'data':vector_data, 'target':Target}
    return X

In [10]:
D=pickle.load(open('imdbFullPorter.p','rb'))
Data=D['data']
Target=D['target']
Xtrain, Xtest, ytrain, ytest = train_test_split(Data, Target, test_size=0.3, random_state=42)
Xtrain2, Xtest2, ytrain, ytest , voc2 = text_to_vector(Xtrain, Xtest, ytrain, ytest, n_grams = (1,3))

Converting text into vectors.
Writing data.
Data written successfully
Finished conversion.



O método *`unpack`* foi criado de modo a simplificar o código e de modo a evitar a repetição de código. Este método apenas recebe um dicionário com Data e Target e retorna estes valores.


In [11]:
"""Método que retorna as keys existentes no ficheiro"""
def unpack(X):
    
    x=X['data']
    y=X['target']
    return x,y

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
## Classificador Binário <a name="binario"/>

Após termos efetuado a vetorização do texto, passamos para a classificação, em binária, dos dados usados. Escolhidos três tipos de classificadores, elaboramos um método para cada classificador, para ficar a saber qual aquele que apresenta melhores resultados.

Como primeiro passo, iremos importar as bibliotecas necessárias, e os seus métodos que cada biblioteca contém.


In [12]:
import pickle
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

O primeiro classificador que decidimos estudar foi o *`logistic regression`*. Para isso criamos o método *`logistic_regression(Xtrain,Xtest,y1,y2, penaltyType = 'l1',cValue = None, solverType = 'liblinear', save_string="lr")`* que recebe como argumentos os dados de treino e teste, ambos dividos pela vetorização, e passado como argumento o tipo de penalização que este classificador recebe(neste caso o lasso, ou seja 'l1', podendo ser alterado para ridge, ou seja 'l2'), o valor do coeficiente de correlação (neste caso ficou a None, visto que foi necessário para obter o melhor parâmetro), e o tipo de solver(que neste caso, obtamos por deixar como sendo 'liblinear').

Para verificar qual o melhor parâmetro, criamos a condição *`if cValue is None`*. Caso esta condição seja verdadeira, verificamos o tipo de penalização para este classificador:

* Caso o tipo de penalização seja lasso, ou seja, 'l1', criamos um dicionário de nome *`parametros`*, onde contém os diferentes valores do coeficiente de correlação, e os diferentes tipos de solvers que serão usados, e no final usamos o método *`logistic regression`* para aplicar consoante estes valores.
* Caso o tipo de penalização seja ridge, ou seja, 'l2', criamos um dicionário de nome *`parametros`*, onde contém os diferentes valores do coeficiente de correlação, e os diferentes tipos de solvers que serão usados, e no final usamos o método *`logistic regression`* para aplicar consoante estes valores.
* Ainda dentro da condição, caso não seja escolhido um valor do coeficiente de correlação, usamos o método *`GridSearchCV`* que permite obter o melhor parâmetro, passando este tipo de classificador.

Caso a condição não seja verdadeira, aplicamos o método *`logistic regression`* consoante os valores que passamos nos argumentos no início deste método.

No final é calculado o valor do score, tanto do treino como do teste, obtendo diferentes resultados, e tirar conclusões de qual o melhor solver e o melhor parâmetro a ser usado. Também optamos por imprimir a matriz de confusão, vendo quantos tipos de verdadeiros/falsos positivos/negativos podiamos obter. E no final, para testar que estamos a usar este classificador como sendo binário, imprimimos a matriz. 

In [13]:
"""
Método que calcula regressão logística, consoante o tipo de penalização(lasso - l1 ou ridge - l2)
e os diferentes tipos de solvers que existem
"""
def logistic_regression(Xtrain,Xtest,y1,y2, penaltyType = 'l1',cValue = None, solverType = 'liblinear',
                        save_string="lr"):
    
    gs = None
    tipoRegressao = "Lasso" if penaltyType == "l1" else "Ridge\n" 
    gs_best_parameter = None
    print("Initializing logistic regression ","Lasso" if penaltyType == "l1" else "Ridge\n")
    if cValue is None:
        if penaltyType == "l1":
            parametros = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'solver':('liblinear','saga')
                          }
            lr = LogisticRegression(penalty=penaltyType,C= cValue,solver = solverType,max_iter=5000)
        elif penaltyType == "l2":
            parametros={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                        'solver' : ('saga','sag', 'lbfgs', 'newton-cg')
                        }
            lr = LogisticRegression(penalty = penaltyType,C= cValue,solver = solverType,max_iter = 5000)
        gs = GridSearchCV(lr, parametros)
        gs.fit(Xtrain,y1)
        gs_best_parameter = gs.best_params_
        print("Best parameter: ", str(gs_best_parameter))
    else:
        gs = LogisticRegression(penalty = penaltyType, solver = solverType, C = cValue, max_iter = 5000)
        gs.fit(Xtrain,y1)
        
    save_data(gs,save_string) 
    print("----", gs)
    scoreTrain = np.round(gs.score(Xtrain, y1) * 100, 3)
    scoreTest = np.round(gs.score(Xtest, y2) * 100, 3)
    
    print("Train success: " + str(np.round(gs.score(Xtrain, y1) * 100, 3)))
    print("Test success: " + str(np.round(gs.score(Xtest, y2) * 100, 3)))
    y2n = gs.predict(Xtest)
    print("Confusion matrix: \n" + str(confusion_matrix(y2, y2n)))
    print("Test Classes: " + str(gs.predict(Xtest)))
    print("Logistic regression ","Lasso" if penaltyType == "l1" else "Ridge",  "Finished ")
    
    return scoreTrain,scoreTest

In [30]:
bin_classify(Xtrain2, Xtest2, ytrain, ytest,
                classifier_type = 'logistic lasso', 
                cValue = 100,
                solverType="saga",
                save_str="bin_lr_lasso_porter")

Binary classification
Inicialização de Regressão Logistica  Lasso
Writing data.
Data written successfully
Train success: 100.0
Test success: 88.725
Confusion matrix: 
[[5417  725]
 [ 628 5230]]
Test Classes: [0. 0. 0. ... 1. 0. 0.]
Logistic regression  Lasso Finished 
Finished binary classification


(100.0, 88.725)

In [50]:
bin_classify(Xtrain2, Xtest2, ytrain, ytest,
                classifier_type = 'logistic ridge', 
                cValue = 10,
                solverType="sag",
                save_str="bin_lr_ridge_porter")

Binary classification
Initializing logistic regression  Ridge

Writing data.
Data written successfully
---- LogisticRegression(C=10, max_iter=5000, solver='sag')
Train success: 99.725
Test success: 90.433
Confusion matrix: 
[[5485  657]
 [ 491 5367]]
Test Classes: [0. 1. 0. ... 1. 0. 1.]
Logistic regression  Ridge Finished 
Finished binary classification


(99.725, 90.433)

In [54]:
D=pickle.load(open('bin_lr_ridge_porter.p','rb'))
gs=D['data']
print(gs)
y2file=gs.predict(Xtest2)
print("Confusion matrix: \n" + str(confusion_matrix(ytest, y2file)))
print("nonzero y2n",np.count_nonzero(y2file))
print("nonzero y2",np.count_nonzero(ytest))

LogisticRegression(C=10, max_iter=5000, solver='sag')
Confusion matrix: 
[[   0    0    0    0    0    0    0    0    0]
 [2452  135    0    0    0    0    0    0    0]
 [1015   89    0    0    0    0    0    0    0]
 [1032  156    0    0    0    0    0    0    0]
 [ 986  277    0    0    0    0    0    0    0]
 [ 181  909    0    0    0    0    0    0    0]
 [ 127 1243    0    0    0    0    0    0    0]
 [  65  991    0    0    0    0    0    0    0]
 [ 118 2224    0    0    0    0    0    0    0]]
nonzero y2n 6024
nonzero y2 12000


O segundo classificador que decidimos estudar foi o *`SVM, Máquina de Suporte Vetorial`*. Para isso criámos o método *`SVM(Xtrain, Xtest, y1, y2, cValue = None, kernelType = None, save_string="svm")`* que recebe como argumentos os dados de treino e teste, ambos dividos pela vetorização, e passado como argumento o tipo de kernel que este classificador recebe o valor do coeficiente de correlação (neste caso ficou a None, visto que foi necessário para obter o melhor parâmetro).

Para verificar qual o melhor parâmetro, criamos a condição *`if cValue is None and kernelType is None`*. Caso esta condição seja verdadeira, criamos um dicionário de nome *`parametros`*, onde contém os diferentes valores do coeficiente de correlação, e os diferentes tipos de kernels que serão usados, e no final usamos o método *`SVC`* para aplicar consoante estes valores. Ainda dentro da condição, caso não seja escolhido um valor do coeficiente de correlação, usamos o método *`GridSearchCV`* que permite obter o melhor parâmetro, passando este tipo de classificador.

Caso a condição não seja verdadeira, aplicamos o método *`SVC`* consoante os valores que passamos nos argumentos no início deste método.

No final é calculado o valor do score, tanto do treino como do teste, obtendo diferentes resultados, e tirar conclusões de qual o melhor solver e o melhor parâmetro a ser usado. Também optamos por imprimir a matriz de confusão, vendo quantos tipos de verdadeiros/falsos positivos/negativos podiamos obter. E no final, para testar que estamos a usar este classificador como sendo binário, imprimimos a matriz. 

In [33]:
"""Método que calcula o SVM(máquina de suporta vetorial), passando um tipo de kernel"""
def SVM(Xtrain, Xtest, y1, y2, cValue = None, kernelType = None, save_string="svm"):
    
    gs = None
    print("Starting SVM")
    if cValue is None and kernelType is None:
        parameters = {
            'C': [0.001, 0.01, 0.1, 1, 10, 100], 
            'kernel' : ('linear', 'rbf', 'sigmoid')
            }
        svm = SVC(gamma = 'auto',max_iter=-1,decision_function_shape='ovr')
        gs = GridSearchCV(svm, parameters)
        gs.fit(Xtrain,y1)
        print("Best parameter: ", str(gs.best_params_))
    else:
        gs = SVC(gamma='auto', max_iter= -1, decision_function_shape='ovr', C = cValue, kernel = kernelType)
        gs.fit(Xtrain,y1)
    
    save_data(gs,save_string)
    print("Train success: " + str(np.round(gs.score(Xtrain, y1) * 100, 3)))
    print("Test success: " + str(np.round(gs.score(Xtest, y2) * 100, 3)))
    y2n = gs.predict(Xtest)
    print("Confusion matrix: \n" + str(confusion_matrix(y2, y2n)))
    print("nonzero y2n",np.count_nonzero(y2n))
    print("nonzero y2",np.count_nonzero(y2))
    
    # print("Test classes: " + str(gs.predict(Xtest)))
    print("SVM concluded")
    return np.round(gs.score(Xtrain, y1) * 100, 3), np.round(gs.score(Xtest, y2) * 100, 3) 


In [38]:
# Determinar melhor parametro para o SVM utilizando menos dados
bin_classify(Xtrain2, Xtest2, ytrain, ytest,
                classifier_type = 'svm', 
                cValue = 1, 
                kernelType = 'linear',
                n_neighborsValue=None,
                save_str='bin_svm_porter')

Binary classification
Starting SVM
Writing data.
Data written successfully
Train success: 98.104
Test success: 90.208
Confusion matrix: 
[[5457  685]
 [ 490 5368]]
nonzero y2n 6053
nonzero y2 5858
SVM concluded
Finished binary classification


(98.104, 90.208)

O terceiro e último classificador que decidimos estudar foi o *`kNN`*. Para isso criamos o método *`kNN(Xtrain, Xtest, y1, y2, n_neighborsValue = 5, save_string='knn')`* que recebe como argumentos os dados de treino e teste, ambos dividos pela vetorização, e passado como argumento a quantidade de vizinhos, neste caso, para teste, optamos por deixar com 5 vizinhos.

Para verificar qual o melhor parâmetro, criamos a condição *`if n_neighborsValue == None`*. Caso esta condição seja verdadeira, criamos um dicionário de nome *`parameters`*, onde contém os diferentes valores de vizinhança. De seguida, usamos o método *`KNeighborsClassifier()`*. Ainda dentro desta condição, para obter o melhor parâmetro(neste caso, para este classificador, o melhor valor de vizinhança) 
Caso a condição não seja verdadeira, aplicamos o método *`SVC`* consoante os valores que passamos nos argumentos no início deste método.Usamos o método *`GridSearchCV`* que permite obter o melhor parâmetro, passando este tipo de classificador.

No final é calculado o valor do score, tanto do treino como do teste, obtendo diferentes resultados, e tirar conclusões de qual o melhor melhor vizinho a ser usado. Também optamos por imprimir a matriz de confusão, vendo quantos tipos de verdadeiros/falsos positivos/negativos podiamos obter. E no final, para testar que estamos a usar este classificador como sendo binário, imprimimos a matriz de teste. 

In [58]:
"""Método que calcula a vizinhança knn, passando como argumento o número de vizinhos"""
def kNN(Xtrain, Xtest, y1, y2, n_neighborsValue = 5, save_string='knn'):
    gs = None
    print("kNN started.")
    if n_neighborsValue == None:
        # Parametros a testar
        parameters = {
                'n_neighbors': [ 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 
                                100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 
                                200, 205, 210, 215, 220, 225, 230, 235, 240, 245, 250, 255, 260, 265, 270, 275, 280, 285, 290, 295, 
                                300, 305, 310, 315, 320, 325, 330, 335, 340, 345, 350, 355, 360, 365, 370, 375, 380, 385, 390, 395, 
                                400, 405, 410, 415, 420, 425, 430, 435, 440, 445, 450, 455, 460, 465, 470, 475, 480, 485, 490, 495] 

                }
        knn = KNeighborsClassifier()
        # GridSearch para obter os melhores parametros de classificação
        gs = GridSearchCV(knn, parameters)
        gs.fit(Xtrain, y1)
        print("Melhor parametro: " + str(gs.best_params_))
    else:
        gs = KNeighborsClassifier(n_neighbors = n_neighborsValue)
        gs.fit(Xtrain, y1)
    
    save_data(gs,save_string)
    print("Train success: " + str(np.round(gs.score(Xtrain, y1) * 100, 3)))
    print("Test success: " + str(np.round(gs.score(Xtest, y2) * 100, 3)))
    y2n = gs.predict(Xtest)
    print("Confusion matrix: \n" + str(confusion_matrix(y2, y2n)))
    print("Test classes: " + str(gs.predict(Xtest)))
    print("kNN Classifier concluded.")
    

In [61]:
bin_classify(Xtrain2, Xtest2, ytrain, ytest,
                classifier_type = 'knn', 
                n_neighborsValue=200,
                save_str='bin_knn_nn200_porter')

Binary classification
kNN started.
Writing data.
Data written successfully
Train success: 82.943
Test success: 82.367
Confusion matrix: 
[[5191  951]
 [1165 4693]]
Test classes: [0. 0. 0. ... 1. 0. 1.]
kNN Classifier concluded.
Finished binary classification


In [63]:
bin_classify(Xtrain2, Xtest2, ytrain, ytest,
                classifier_type = 'knn', 
                n_neighborsValue=300,
                save_str='bin_knn_nn300_porter')

Binary classification
kNN started.
Writing data.
Data written successfully
Train success: 82.893
Test success: 82.475
Confusion matrix: 
[[5269  873]
 [1230 4628]]
Test classes: [0. 0. 0. ... 1. 0. 1.]
kNN Classifier concluded.
Finished binary classification


Para finalizar a parte que diz respeito à classificação binária, foi realizado o método *`bin_classify(Xtrain,Xtest,y1,y2,classifier_type = 'logistic lasso', cValue = None, solverType = None, kernelType = None,n_neighborsValue=None,save_str=None)`*. Dentro deste método, fazemos a conversão dos dados para binário, através da seguinte condição:*` y1 = (y1 >= 7) * 1.0`* e *` y2 = (y2 >= 7) * 1.0`*. Para finalizar, criamos diferentes condições para verificar qual o tipo de classificador que queremos correr. Ao ser chamado este método, sempre que o tipo de classificador seja "logistic lasso", irá ser corrido o método que foi criado para a regressão logísica, neste caso, para o lasso. O procedimento para correr os outros classificadores é idêntico. 

In [35]:
"""Método que retorna os resulados do treino e do teste, consoante a classificação (em binário) escolhida"""
def bin_classify(Xtrain,Xtest,y1,y2,
                classifier_type = 'logistic lasso', 
                cValue = None, 
                solverType = None, 
                kernelType = None,
                n_neighborsValue=None,
                save_str=None):
    
    y1 = (y1 >= 7) * 1.0
    y2 = (y2 >= 7) * 1.0
    print("Binary classification")    
    classifier_type = classifier_type.lower() # evitar erros de capitalização
    resultado = 0
    
    if classifier_type == 'logistic lasso': result = logistic_regression(Xtrain, Xtest, y1, y2,'l1', cValue, solverType, save_str)
    if classifier_type == 'logistic ridge': result = logistic_regression(Xtrain, Xtest, y1, y2,'l2', cValue, solverType, save_str)
    if classifier_type == 'svm': result = SVM(Xtrain, Xtest, y1, y2,cValue,kernelType, save_str)
    if classifier_type == 'knn': result = kNN(Xtrain, Xtest, y1, y2, n_neighborsValue, save_str)
    print("Finished binary classification")
    return result



O método *`binClassify`* recebe o dicionário com os dados vetorizados resultante da aplicação do método *`text2vector`*.
Neste método é evocado o método *`unpack`* para obter os dados e as classes a que estes dados pertencem. Seguidamente é obtido o classificador binário previamente treinado que obteve melhores resultados, o SVM, e é realizado o *`predict`* sobre os dados que são fornecidos. Por fim é retornado um array com o resultado deste *`predict`*.


In [21]:
"""Método que classifica o ficheiro passado em binário, com o melhor clasificador usado"""
def binClassify(X):
    x, y = unpack(X)
    fName = "bin_svm_porter.p"
    D = pickle.load(open(fName, 'rb'))
    gs = D['data']
    yBin = gs.predict(x)
    return yBin

In [23]:
D=pickle.load(open('imdbFullPorter.p','rb'))
X = text2vector(D)
x=X['data']
y=X['target']
y2=(y >= 7) * 1.0
ym=binClassify(X)
print("Confusion matrix: \n" + str(confusion_matrix(y2, ym)))

Start text clearing
Text clearing finished
Start stemming:  Porter Stemmer

Stemming finished.

Confusion matrix: 
[[19171  1039]
 [  731 19059]]


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
## Classificador Multinomial <a name="multiclass"/>

Após termos efetuado a vetorização do texto, e com a classificação binária tratada passamos para a classificação, em multiclass, dos dados usados. Foram escolhidos os mesmos três tipos de classificadores, elaboramos um método para cada classificador, para ficar a saber qual aquele que apresenta melhores resultados.

O primeiro classificador que decidimos estudar foi o *`logistic regression`*. Para isso criamos o método *`logistic_regression(Xtrain,Xtest,y1,y2, penaltyType = 'l1',cValue = None, solverType = 'liblinear', save_string="lr")`* que recebe como argumentos os dados de treino e teste, ambos dividos pela vetorização, e passado como argumento o tipo de penalização que este classificador recebe(neste caso o lasso, ou seja 'l1', podendo ser alterado para ridge, ou seja 'l2'), o valor do coeficiente de correlação (neste caso ficou a None, visto que foi necessário para obter o melhor parâmetro), e o tipo de solver(que neste caso, obtamos por deixar como sendo 'liblinear').

Para verificar qual o melhor parâmetro, criamos a condição *`if cValue is None`*. Caso esta condição seja verdadeira, verificamos o tipo de penalização para este classificador:

* Caso o tipo de penalização seja lasso, ou seja, 'l1', criamos um dicionário de nome *`parametros`*, onde contém os diferentes valores do coeficiente de correlação, e os diferentes tipos de solvers que serão usados, e no final usamos o método *`logistic regression`* para aplicar consoante estes valores.
* Caso o tipo de penalização seja ridge, ou seja, 'l2', criamos um dicionário de nome *`parametros`*, onde contém os diferentes valores do coeficiente de correlação, e os diferentes tipos de solvers que serão usados, e no final usamos o método *`logistic regression`* para aplicar consoante estes valores.
* Sendo que estamos a estudar com um classificador multiclass, foi necessário passar o argumento *`multi_class='multinomial'`* quando é usado o método *`logistic regression`*, tendo a garantia que a classificação que obtemos no final, é em multiclass.
* Ainda dentro da condição, caso não seja escolhido um valor do coeficiente de correlação, usamos o método *`GridSearchCV`* que permite obter o melhor parâmetro, passando este tipo de classificador.


Caso a condição não seja verdadeira, aplicamos o método *`logistic regression`* consoante os valores que passamos nos argumentos no início deste método.

No final é calculado o valor do score, tanto do treino como do teste, obtendo diferentes resultados, e tirar conclusões de qual o melhor solver e o melhor parâmetro a ser usado. Também optamos por imprimir a matriz de confusão, vendo quantos tipos de verdadeiros/falsos positivos/negativos podiamos obter. E no final, para testar que estamos a usar este classificador como sendo binário, imprimimos a matriz. 

In [31]:
"""
Método que calcula regressão logística, consoante o tipo de penalização(lasso - l1 ou ridge - l2)
e os diferentes tipos de solvers que existem
"""

def RegressaoLogistica(Xtrain,Xtest,y1,y2, penaltyType = 'l1',cValue = None, solverType = 'saga'):
    
    gs = None
    tipoRegressao = "Lasso" if penaltyType == "l1" else "Ridge\n" 
    gsMelhorParametro = None
    print("Inicialização de Regressão Logistica ",tipoRegressao)
    if cValue is None:
        if penaltyType == "l1":
            parametros = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
            lr = LogisticRegression(penalty=penaltyType,C= cValue,solver = solverType,max_iter=5000,
                                    multi_class='multinomial')
        elif penaltyType == "l2":
            parametros={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                        'solver' : ('saga','sag', 'lbfgs', 'newton-cg')
                        }
            lr = LogisticRegression(penalty = penaltyType,C= cValue,solver = solverType,max_iter = 5000,
                                        multi_class='multinomial')
        gs = GridSearchCV(lr, parametros)
        gs.fit(Xtrain,y1)
        gsMelhorParametro = gs.best_params_
        print("O melhor parametro é: ", str(gsMelhorParametro))
    else:
        gs = LogisticRegression(penalty = penaltyType, solver = solverType, C = cValue, max_iter = 5000,multi_class='multinomial')
        gs.fit(Xtrain,y1)
    
    save_data(gs,"multi_lasso_porter")
    scoreTrain = np.round(gs.score(Xtrain, y1) * 100, 3)
    scoreTest = np.round(gs.score(Xtest, y2) * 100, 3)
    
    print("Acertos no Treino: " + str(scoreTrain))
    print("Acertos no Teste: " + str(scoreTest))
    y2n = gs.predict(Xtest)
    print("Matriz de Confusão: \n" + str(confusion_matrix(y2, y2n)))
    print("As classes do Teste são: " + str(y2n))
    print("Regressão Logistica",tipoRegressao,solverType, "concluída")
    
    return scoreTrain,scoreTest

In [32]:
print("-----------------------LOGISTIC lasso - saga - 1---------------------------")
multi_classify(Xtrain2, Xtest2, ytrain, ytest,classifier_type = 'logistic lasso', cValue = 1, solverType = 'saga')

-----------------------LOGISTIC lasso - saga - 1---------------------------
Multiclass classification
Inicialização de Regressão Logistica  Lasso
Writing data.
Data written successfully
Acertos no Treino: 47.643
Acertos no Teste: 42.733
Matriz de Confusão: 
[[2059   85   94  125   27   25    7  165]
 [ 642   73   99  137   30   35    4   84]
 [ 431   74  150  292   58   38   10  135]
 [ 307   53  136  384  105  106   19  153]
 [  73    9   29  115  253  238   48  325]
 [  81   12   25   69  204  293   54  632]
 [  50    3    8   44   84  146   46  675]
 [  96    5   17   37   74  166   77 1870]]
As classes do Teste são: [ 4  7  1 ... 10  1  1]
Regressão Logistica Lasso saga concluída
Multiclass classification finished.


(47.643, 42.733)

O segundo classificador que decidimos estudar foi o *`SVM, Máquina de Suporte Vetorial`*. Para isso criamos o método *`SVM(Xtrain, Xtest, y1, y2, cValue = None, kernelType = None, save_string="svm")`* que recebe como argumentos os dados de treino e teste, ambos dividos pela vetorização, e passado como argumento o tipo de kernel que este classificador recebe o valor do coeficiente de correlação (neste caso ficou a None, visto que foi necessário para obter o melhor parâmetro).

Para verificar qual o melhor parâmetro, criamos a condição *`if cValue is None and kernelType is None`*. Caso esta condição seja verdadeira, criamos um dicionário de nome *`parametros`*, onde contém os diferentes valores do coeficiente de correlação, e os diferentes tipos de kernels que serão usados, e no final usamos o método *`SVC`* para aplicar consoante estes valores. Ainda dentro da condição, caso não seja escolhido um valor do coeficiente de correlação, usamos o método *`GridSearchCV`* que permite obter o melhor parâmetro, passando este tipo de classificador.
Para ter a garantia que este classificador retorna os resultados em multiclass, foi passado o argumento *`decision_function_shape='ovo'`*, ou seja, one-versus-one.

Caso a condição não seja verdadeira, aplicamos o método *`SVC`* consoante os valores que passamos nos argumentos no início deste método.

No final é calculado o valor do score, tanto do treino como do teste, obtendo diferentes resultados, e tirar conclusões de qual o melhor solver e o melhor parâmetro a ser usado. Também optamos por imprimir a matriz de confusão, vendo quantos tipos de verdadeiros/falsos positivos/negativos podiamos obter. E no final, para testar que estamos a usar este classificador como sendo binário, imprimimos a matriz. 

In [27]:
"""Método que calcula o SVM(máquina de suporta vetorial), passando um tipo de kernel"""
def SVM(Xtrain, Xtest, y1, y2, cValue = None, kernelType = None):
    
    gs = None
    print("Starting SVM")
    if cValue is None and kernelType is None:
        parametros = {
            'C': [0.001, 0.01, 0.1, 1, 10, 100], 
            'kernel' : ('linear', 'rbf', 'sigmoid')
            }
        svm = SVC(gamma = 'auto',max_iter=5000,decision_function_shape='ovo')
        gs = GridSearchCV(svm, parametros)
        gs.fit(Xtrain,y1)
        print("O melhor parametro é: ", str(gs.best_params_))
    else:
        gs = SVC(gamma='auto', max_iter= 5000, decision_function_shape='ovo', C = cValue, kernel = kernelType)
        gs.fit(Xtrain,y1)
        
    # save_data(gs,"multi_svm_porter")    
    print("Train success: " + str(np.round(gs.score(Xtrain, y1) * 100, 3)))
    print("Test success: " + str(np.round(gs.score(Xtest, y2) * 100, 3)))
    y2n = gs.predict(Xtest)
    print("Confusion matrix: \n" + str(confusion_matrix(y2, y2n)))
    print("Test classes: " + str(gs.predict(Xtest)))
    print("SVM concluded")
    return np.round(gs.score(Xtrain, y1) * 100, 3), np.round(gs.score(Xtest, y2) * 100, 3) 

O terceiro e último classificador que decidimos estudar foi o *`kNN`*. Para isso criamos o método *`kNN(Xtrain, Xtest, y1, y2, n_neighborsValue = 5, save_string='knn')`* que recebe como argumentos os dados de treino e teste, ambos dividos pela vetorização, e passado como argumento a quantidade de vizinhos, neste caso, para teste, optamos por deixar com 5 vizinhos.

Para verificar qual o melhor parâmetro, criamos a condição *`if n_neighborsValue == None`*. Caso esta condição seja verdadeira, criamos um dicionário de nome *`parameters`*, onde contém os diferentes valores de vizinhança. De seguida, usamos o método *`KNeighborsClassifier()`*. Ainda dentro desta condição, para obter o melhor parâmetro(neste caso, para este classificador, o melhor valor de vizinhança) 
Caso a condição não seja verdadeira, aplicamos o método *`SVC`* consoante os valores que passamos nos argumentos no início deste método.Usamos o método *`GridSearchCV`* que permite obter o melhor parâmetro, passando este tipo de classificador.

No final é calculado o valor do score, tanto do treino como do teste, obtendo diferentes resultados, e tirar conclusões de qual o melhor melhor vizinho a ser usado. Também optamos por imprimir a matriz de confusão, vendo quantos tipos de verdadeiros/falsos positivos/negativos podiamos obter. E no final, para testar que estamos a usar este classificador como sendo binário, imprimimos a matriz de teste. 

In [2]:
"""Método que calcula a vizinhança knn, passando como argumento o número de vizinhos"""
def kNN(Xtrain,Xtest,y1,y2,n_neighborsValue = 5):
    
    gs = None
    print("kNN started.")
    if n_neighborsValue == None:
        parameters = {
                    'n_neighbors': [ 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 
                                    100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 
                                    200, 205, 210, 215, 220, 225, 230, 235, 240, 245, 250, 255, 260, 265, 270, 275, 280, 285, 290, 295, 
                                    300, 305, 310, 315, 320, 325, 330, 335, 340, 345, 350, 355, 360, 365, 370, 375, 380, 385, 390, 395, 
                                    400, 405, 410, 415, 420, 425, 430, 435, 440, 445, 450, 455, 460, 465, 470, 475, 480, 485, 490, 495] 
    
                    }
        knn = KNeighborsClassifier()
        gs = GridSearchCV(knn, parameters)
        gs.fit(Xtrain,y1)
    else: 
        gs = KNeighborsClassifier(n_neighbors = n_neighborsValue)
        gs.fit(Xtrain,y1)
    
    # save_data(gs,"multi_knn_porter")
    print("Train success: " + str(np.round(gs.score(Xtrain, y1) * 100, 3)))
    print("Test success: " + str(np.round(gs.score(Xtest, y2) * 100, 3)))
    y2n = gs.predict(Xtest)
    print("Confusion matrix: \n" + str(confusion_matrix(y2, y2n)))
    print("Test classes: " + str(gs.predict(Xtest)))
    print("kNN Classifier concluded.")

Para finalizar a parte que diz respeito à classificação binária, foi realizado o método *`multi_classify(Xtrain,Xtest,y1,y2,classifier_type = 'logistic lasso', cValue = None, solverType = None, kernelType = None):`*. Dentro deste método, criamos diferentes condições para verificar qual o tipo de classificador que queremos correr. Ao ser chamado este método, sempre que o tipo de classificador seja "logistic lasso", irá ser corrido o método que foi criado para a regressão logísica, neste caso, para o lasso. O procedimento para correr os outros classificadores é idêntico. 

In [3]:
"""Método que retorna os resultados do treino e do teste, consoante a classificação (em multiclass) escolhida"""
def multi_classify(Xtrain,Xtest,y1,y2,
                classifier_type = 'logistic lasso', 
                cValue = None, 
                solverType = None, 
                kernelType = None):
    

    print("Multiclass classification")    
    classifier_type = classifier_type.lower()
    result = 0
    
    if classifier_type == 'logistic lasso': result = RegressaoLogistica(Xtrain, Xtest, y1, y2,'l1', cValue, solverType)
    if classifier_type == 'logistic ridge': result = RegressaoLogistica(Xtrain, Xtest, y1, y2,'l2', cValue, solverType)
    if classifier_type == 'svm': resultado = SVM(Xtrain, Xtest, y1, y2,cValue, kernelType)
    if classifier_type == 'knn': resultado = kNN(Xtrain, Xtest, y1, y2, solverType)

    print("Multiclass classification finished.")
    return result


O método *`multiClassify`* recebe o dicionário com os dados vetorizados resultante da aplicação do método *`text2vector`*.
Neste método é evocado o método *`unpack`* para obter os dados e as classes a que estes dados pertencem. Seguidamente é obtido o classificador multiclasse previamente treinado que obteve melhores resultados, o regressor logístico com o classificador ridge, e é realizado o *`predict`* sobre os dados que são fornecidos. Por fim é retornado um array com o resultado deste *`predict`*, sendo que desta feita o array é composto por valores de 1 a 4 e de 7 a 10.


In [7]:
"""Método que classifica o ficheiro, em multiclass, com o melhor classificador escolhido"""
def multiClassify(X):
    
    x, y = unpack(X)
    fName= "multi_ridge_porter.p"
    D = pickle.load(open(fName, 'rb'))
    gs = D['data']
    yMulti = gs.predict(x)
    return yMulti

In [17]:
D=pickle.load(open('imdbFullPorter.p','rb'))
X = text2vector(D)
x=X['data']
y=X['target']
ym=multiClassify(X)
print("Confusion matrix: \n" + str(confusion_matrix(y, ym)))

Start text clearing
Text clearing finished
Start stemming:  Porter Stemmer

Stemming finished.

Confusion matrix: 
[[7755   55   94  156   19   35    7  231]
 [1677 1342  124  254   35   53    8  197]
 [1154   53 1885  383   78   69   11  290]
 [ 758   36  154 2711  132  105   21  328]
 [ 174    7   49  156 2136  297   36  889]
 [ 176    9   25   96  227 2508   51 1437]
 [ 121    6   20   55  136  291 1227 1756]
 [ 181    9   12   53  122  212   45 7271]]


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
## Regressão Linear <a name="linear"/>

Um dos tópicos escolhis foi a regressão linear. O objetivo deste tipo de regressão consiste em  treinar e avaliar um modelo de regressão linear e comparar os resultados obtidos no problema de classificação multi-classe, com os da classificação binária.

Este tipo de regressão linear consiste numa equação para se estimar a condicional (valor esperado) de uma variável y, dados os valores de algumas outras variáveis x.

A regressão linear é chamada "linear" porque se considera que a relação da resposta às variáveis é uma função linear de alguns parâmetros. Os modelos de regressão que não são uma função linear dos parâmetros se chamam modelos de regressão não-linear.

Como sempre, importamos as bibliotecas necessárias para a realização dos métodos acerca deste tópico.

In [3]:
import numpy as np
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import PolynomialFeatures

O primeiro método que decidimos criar foi o *`RegressaoLinear(Xtrain,Xtest,y1,y2)`* que recebe como argumentos os dados de treino e teste, ambos dividos pela vetorização.

Este método apenas calcula a regressão linear, chamando o método *`LinearRegression`*, que permite calcular a regressão linear, passando os dados de treino.

No final é calculado o valor do coeficiente R2, tanto do treino como do teste, obtendo diferentes resultados, e tirar conclusões.

In [4]:
"""Método que calcula a regressão linear"""
def RegressaoLinear(Xtrain,Xtest,y1,y2):
    
    print("Inicio da regressão linear \n")
    lr = LinearRegression().fit(Xtrain,y1)
    scoreTrain = np.round(lr.score(Xtrain,y1),2)
    scoreTest = np.round(lr.score(Xtest,y2),2)
    print("Resultado do coeficiente R2 da classe Treino: ",str(scoreTrain))
    print("Resultado do coeficiente R2 da classe Treino: ",str(scoreTest))
    

De seguida, prodecemos à regressão linear, no que diz respeito ao lasso(visto que é necessário era necessário testar os dados com um modelo de regressão linear). Para isso criamos o método *`RegressaoLinearLasso(Xtrain,Xtest,y1,y2,alpha = None)`* que recebe como argumentos os dados de treino e teste, ambos dividos pela vetorização, e um valor de alpha, usando os mesmo valores que foram passados como coeficente de correlação nos classificadores anteriores.

Para verificar qual o melhor valor do alpha, criamos a condição *`if alpha is None`*. Caso esta condição seja verdadeira, criamos um dicionário, de nome *`parametros`* que contém os mesmos valores que foram passados nos classificadores passados. De seguida usamos o método *`Lasso()`* para aplicar a regressão linear, consoante o classificador usado, e no final, usar o método *`GridSearchCV`* para determinar qual o melhor parâmetro, neste caso, o melhor *`alpha`*.

Caso a condição não seja verdadeira, aplicamos o método *`Lasso`* consoante os valores que passamos nos argumentos no início deste método.

No final é calculado o valor do coeficiente R2, tanto do treino como do teste, obtendo diferentes resultados, e tirar conclusões.

In [9]:
"""Método que calcula a regressao linear com Lasso"""
def RegressaoLinearLasso(Xtrain,Xtest,y1,y2,alpha = None):
    
    gs = None
    print("Inicio do Lasso \n")
    if alpha is None:
        print("Yay entrei no none")
        parametros = {'alpha':[0.001,0.01,0.1,1,10,10,1000]}
        lasso = Lasso(max_iter=5000)
        gs = GridSearchCV(lasso,parametros).fit(Xtrain,y1)
        print("Melhor parametro: " + str(gs.best_params_))
    else:
        gs = Lasso(alpha,max_iter=5000).fit(Xtrain,y1)
    
    scoreTreino = np.round(gs.score(Xtrain,y1),2)
    scoreTeste = np.round(gs.score(Xtest,y2),2)
    
    print("Resultado do coeficiente R2 da classe Treino: ",scoreTreino)
    print("Resultado do coeficiente R2 da classe Teste: ",scoreTeste)
    
    return scoreTreino,scoreTeste


De seguida, efetuamos o mesmo tipo de regressão linear, mas neste caso, fomos calcular para o método*`Ridge`*. Para isso criamos o método *`RegressaoLinearRidge(Xtrain,Xtest,y1,y2,alpha = None)`* que recebe como argumentos os dados de treino e teste, ambos dividos pela vetorização, e um valor de alpha, usando os mesmo valores que foram passados como coeficente de correlação nos classificadores anteriores.

Para verificar qual o melhor valor do alpha, criamos a condição *`if alpha is None`*. Caso esta condição seja verdadeira, criamos um dicionário, de nome *`parametros`* que contém os mesmos valores que foram passados nos classificadores passados. De seguida usamos o método *`Ridge()`* para aplicar a regressão linear, consoante o classificador usado, e no final, usar o método *`GridSearchCV`* para determinar qual o melhor parâmetro, neste caso, o melhor *`alpha`*.

Caso a condição não seja verdadeira, aplicamos o método *`Ridge`* consoante os valores que passamos nos argumentos no início deste método.

No final é calculado o valor do coeficiente R2, tanto do treino como do teste, obtendo diferentes resultados, e tirar conclusões.

In [10]:
"""Método que calcula a regressao linear com Ridge"""
def RegressaoLinearRidge(Xtrain,Xtest,y1,y2,alpha = None):
    
    gs = None
    print("Inicio do Ridge \n")
    
    if alpha is None:
        parametros = {'alpha':[0.001,0.01,0.1,1,10,10,1000]}
        ridge = Ridge(max_iter=5000)
        gs = GridSearchCV(ridge,parametros).fit(Xtrain,y1)
        print("Melhor parametro: " + str(gs.best_params_))
    else:
        gs = Ridge(alpha,max_iter=5000).fit(Xtrain,y1)
    
    scoreTreino = np.round(gs.score(Xtrain,y1),2)
    scoreTeste = np.round(gs.score(Xtest,y2),2)
    
    print("Resultado do coeficiente R2 da classe Treino: ",scoreTreino)
    print("Resultado do coeficiente R2 da classe Teste: ",scoreTeste)
    
    return scoreTreino,scoreTeste

De seguida, efetuamos regressão linear, mas para um modelo não linear. Para isso criamos o método *`MaquinaSuporteVetorialLinear(Xtrain,Xtest,y1,y2,cValue = None,kernelType = 'rbf')`* que recebe como argumentos os dados de treino e teste, ambos dividos pela vetorização, e passado como argumento o tipo de kernel que este classificador recebe o valor do coeficiente de correlação (neste caso ficou a None, visto que foi necessário para obter o melhor parâmetro).

Para verificar qual o melhor parâmetro, criamos a condição *`if cValue is None and kernelType is None`*. Caso esta condição seja verdadeira, criamos um dicionário de nome *`parametros`*, onde contém os diferentes valores do coeficiente de correlação, e os diferentes tipos de kernels que serão usados, e no final usamos o método *`SVC`* para aplicar consoante estes valores. Ainda dentro da condição, caso não seja escolhido um valor do coeficiente de correlação, usamos o método *`GridSearchCV`* que permite obter o melhor parâmetro, passando este tipo de classificador.

Caso a condição não seja verdadeira, aplicamos o método *`SVC`* consoante os valores que passamos nos argumentos no início deste método.

No final é calculado o valor do score, tanto do treino como do teste, obtendo diferentes resultados, e tirar conclusões de qual o melhor solver e o melhor parâmetro a ser usado. 

In [11]:
"""Método que calcula o SVR"""
def MaquinaSuporteVetorialLinear(Xtrain,Xtest,y1,y2,cValue = None,kernelType = 'rbf'):
    
    gs = None
    print("Inicio do SVR \n")
    if cValue == None and kernelType == None:
        parametros = {
            'C': [0.001, 0.01, 0.1, 1, 10, 100], 
            'kernel' : ('linear', 'rbf', 'poly')
            }
        svr = SVR(gamma = 'auto',max_iter=-1)
        gs = GridSearchCV(svr,parametros).fit(Xtrain,y1)
        print("Melhor parametro: " + str(gs.best_params_))
    else:
        gs = SVR(gamma = 'auto',max_iter=-1, C = cValue,kernel = kernelType).fit(Xtrain,y1)
    scoreTreino = np.round(gs.score(Xtrain,y1),2)
    scoreTeste = np.round(gs.score(Xtest,y2),2)
    
    print("Resultado do coeficiente R2 da classe Treino: ",scoreTreino)
    print("Resultado do coeficiente R2 da classe Teste: ",scoreTeste)
    
    return scoreTreino,scoreTeste

Para finalizar a parte que diz respeito à regressão linear, foi realizado o método *`classificadorRegressao(Xtrain,Xtest,y1,y2,binario = True, tipoClassificador = "regressao linear",alpha = None,
 cValue = None,kernelType = 'rbf')`*. Dentro deste método, criamos diferentes condições para verificar qual o tipo de classificador que queremos correr. Passando um valor booleano como argumento deste método,caso seja *`True`*, ele efetua uma classificação de regressão linear como sendo binária, mas caso essa condição seja *`False`*, ele executa o método como sendo uma regressão linear em multiclass. Ao ser chamado este método, sempre que o tipo de classificador seja "regressao linear", irá ser corrido o método que foi criado para a regressão logísica, neste caso, para o lasso. O procedimento para correr os outros classificadores é idêntico. 

In [12]:
"""Método que retorna os resulados do treino e do teste, consoante a classificação escolhida"""
def classificadorRegressao(Xtrain,Xtest,y1,y2,binario = True, 
                           tipoClassificador = "regressao linear",
                           alpha = None,
                           cValue = None,
                           kernelType = 'rbf'):
    
    if binario:
        y1 = (y1 >= 7) * 1.0
        y2 = (y2 >= 7) * 1.0
        
    tipoClassificador = tipoClassificador.lower()
    resultado = 0
    
    if tipoClassificador == 'regressao linear': resultado = RegressaoLinear(Xtrain, Xtest, y1, y2)
    elif tipoClassificador == 'regressao ridge': resultado = RegressaoLinearRidge(Xtrain, Xtest, y1, y2,alpha)
    elif tipoClassificador == 'regressao lasso': resultado = RegressaoLinearLasso(Xtrain, Xtest, y1, y2,alpha)
    elif tipoClassificador == 'svr': resultado = MaquinaSuporteVetorialLinear(Xtrain, Xtest, y1, y2, cValue,kernelType)
    print("Fim da classificação Regressao Linear \n")
    return resultado

In [ ]:
fN = 'imdbFullPorter.p'

D = pickle.load(open(fN,'rb')) 
D.keys() 
dados = D['data']
target = D['target']

X1, X2, y1, y2 = train_test_split(dados,target,test_size=0.3,random_state = 42)
x_train, x_test, ytrain, ytest , voc2 = text_to_vector(X1, X2, y1, y2, n_grams = (1,3))
#----------------------------Regressao Linear---------------------
#print("-----------------------Regressao Linear(bin)--------------------------")
#classificadorRegressao(x_train,x_test,y1,y2,binario= True, tipoClassificador = 'regressao linear',alpha = None)
#print()
#print("-----------------------Regressao Linear-Lasso(bin)--------------------------")
#classificadorRegressao(x_train,x_test,y1,y2,binario= True, tipoClassificador = 'regressao lasso',alpha = None)
#print()
#print("-----------------------Regressao Linear-ridge(bin)--------------------------")
#classificadorRegressao(x_train,x_test,y1,y2,binario= True, tipoClassificador = 'regressao ridge',alpha = None)
#print()
#print("-----------------------Regressao Linear-svr(bin)--------------------------")
#classificadorRegressao(x_train,x_test,y1,y2,binario= True, tipoClassificador = 'svr',cValue = None, kernelType=None)


#print("-----------------------Regressao Linear--------------------------")
#classificadorRegressao(x_train,x_test,y1,y2,binario= False, tipoClassificador = 'regressao linear',alpha = None)
#print()
#print("-----------------------Regressao Linear-Lasso(--------------------------")
#classificadorRegressao(x_train,x_test,y1,y2,binario= False, tipoClassificador = 'regressao lasso',alpha = None)
#print()
#print("-----------------------Regressao Linear-ridge--------------------------")
#classificadorRegressao(x_train,x_test,y1,y2,binario= False, tipoClassificador = 'regressao ridge',alpha = None)
#print()
#print("-----------------------Regressao Linear-svr--------------------------")
#classificadorRegressao(x_train,x_test,y1,y2,binario= False, tipoClassificador = 'svr',cValue = None, kernelType='None')

Para poupar tempo de processamento, decidimos apresentar os resultados nas figuras a seguir, no que diz respeito à regressão linear. Depois de ser feita a classificação para a regressão linear, os resultados obtidos podem ser encontrados nas figuras a seguir. A figura à esquerda permite obter os resultados, quando passado pela regressão linear, mas em binário. Já a figura na direita, foram os resultados obtidos da regressão linear, mas em multiclass.


<img src="https://i.imgur.com/2lizGn1.png" width="400" height="400" style="float:left">
<img src="https://i.imgur.com/D1sZu7J.png" width="400" height="400" style="float:right">

<img src="https://i.imgur.com/sh4w9d6.png" width="400" height="400" style="float:left">
<img src="https://i.imgur.com/0LhRP0P.png" width="400" height="400" style="float:right">

Como se pode observar, nas figuras acima, conseguimos obter o melhor parâmetro alpha, assim como os resultados do treino e do teste. Analisando as imagens, podemos reparar que, comparando os dois tipos de classificadores, os resultados foram idênticos. A única diferença é que o classificador que não está a ser como binário, na regressão linear, lasso, apresenta melhores resultados que o binário. O mesmo se pode dizer o SVR, pois apresenta uma percentagem superior, ao contrário do binário.

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
## Dimensão do Vocabulário<a name="dimensao"/>

O outro tópico escolhido foi a dimensão do vocabulário. O objetivo deste tópico é investigar a influência do tamanho do dicionário(dimensão dos dados) no desempenho de um discriminante logístico no problema de classificação binária.  
Começamos por construir um dicionário(neste caso com 30000 dimensões) aplicando stemming(neste caso, iremos usar mais uma ver, o stemm Porter) e usada a regularização lasso. Para tirar breves conclusões deste tópico, iremos reduzir o tamanho do nosso dicionário, e ver até onde ele consegue processar os dados.

Foi criado o método *`LassoRegression(Xtrain,Xtest,y1,y2,penaltyType = "l1",cValue = None)`* que recebe como argumentos os dados de treino e teste, ambos dividos pela vetorização, o tipo de penalização (neste caso, lasso) e um valor do coeficiente de correlação). 

Dentro deste método, recorremos ao tipo de solver "saga", visto que como tamos a tratar de muitos dados, é o melhor que consegue lidar com uma grande quantidade de dados processados.

De seguida, guardamos os valores dos coeficiente, e efetuamos um sort desses coeficientes, usando o método *`np.argsort`*. No final deste método, é feito prints, do número de erros, através da expressão *`np.sum(w!=0)`* , e feito também, o print da matriz de confusão e os acertos de treino e de teste.

In [ ]:
"""Método que calcula a regressão, em lasso"""
def LassoRegression(Xtrain,Xtest,y1,y2,penaltyType = "l1",cValue = None):
    
    lr = LogisticRegression(penalty = "l1",solver = 'saga', C = cValue).fit(Xtrain,y1)
    w =lr.coef_
    idx = np.argsort(w)
    w = w.squeeze()
    idx = idx.ravel()
    scoreTrain = np.round(lr.score(Xtrain, y1) * 100, 3)
    scoreTest = np.round(lr.score(Xtest, y2) * 100, 3)
    print("Número de Erros: ", np.sum(w!=0))
    print("Número de Acertos: ", np.sum(w==0))
    print("Acertos no Treino: " + str(scoreTrain))
    print("Acertos no Teste: " + str(scoreTest))
    y2n = lr.predict(Xtest)
    print("Matriz de Confusão: \n" + str(confusion_matrix(y2, y2n)))
    print("Fim de regularização Lasso\n")

Foi reaproveitado o rocedemos à realização da divisão do ficheiro, em treino e teste. *`text2Vector(Xtrain,Xtest,n_grams = (2,2),max_featuresV = None)`*, recebendo como argumentos os dados de treino e de teste, passando bigramas (n-gramas de (2,2)), e o max_featuresV.

Recorremos ao método *`TfidfVectorizer`* para aplicar esse conjunto de dados, numa matriz tfidf, passando os n-gramas e o número máximo de features (*`max_featuresV`*).

No final, aplicamos esta transformação, tantos para os dados de treino, como para os dados de teste. No final, retornamos os dados de teste e de treino convertidos, assim como um vocabulário.

In [13]:
"Método que converte um documento, em texto, para uma matriz de vetores(X)"
def text2Vector(Xtrain,Xtest,n_grams = (1,3),max_featuresV = None):
    
    print("Inicialização de converção de texto para vectores.\n")
    tfidf = TfidfVectorizer(ngram_range = n_grams,max_features = max_featuresV).fit(Xtrain)
    voc = tfidf.get_feature_names()
    Xtrain = tfidf.transform(Xtrain)
    Xtest = tfidf.transform(Xtest)
    return Xtrain,Xtest,voc

Para terminar a realização deste tópico, realizamos o método *`classify(Xtrain,Xtest,y1,y2,penaltyType="l1",cValue = None)`*. Dentro deste método, fazemos a conversão dos dados para binário, através da seguinte condição:*` y1 = (y1 >= 7) * 1.0`* e *` y2 = (y2 >= 7) * 1.0`*, e chamamos o método criado no início deste tópico, passando os dados de treino e de teste, assim como o tipo de classificador usado.

In [ ]:
"""Método que retorna uma classificação, em binário"""
def classify(Xtrain,Xtest,y1,y2,penaltyType="l1",cValue = None):
    
    print("Inicialização de classificação\n")
    y1 = (y1 >= 7) * 1.0
    y2 = (y2 >= 7) * 1.0
    
    LassoRegression(Xtrain,Xtest,y1,y2,'l1',cValue)
    print("End of the classify")

In [35]:
fN = 'imdbFullPorter.p'
D = pickle.load(open(fN,'rb')) 
D.keys() 
dados = D['data']
target = D['target']
X1, X2, y1, y2 = train_test_split(dados,target,test_size=0.3,random_state = 42)
x_train, x_test, Vocab = text2Vector(X1, X2,max_featuresV = 150)
classify(x_train, x_test, y1, y2, penaltyType = 'l1', cValue = 1)

Para poupar tempo de processamento, decidimos apresentar os resultados nas figuras a seguir, no que diz respeito aos diferentes valores da dimensão do vocabulário.

Na figura à esquerda, os resultados apresentados, são corridos com 30000 dados. Já na figura a seguir, reduzimos os dados para metade, ou seja, 15000.

<img src="https://i.imgur.com/5DxvTdX.png" width="400" height="400" style="float:left">
<img src="https://i.imgur.com/NGZLC1v.png" width="400" height="400" style="float:right">

Na figura à esquerda, os resultados apresentados, são corridos com 5000 dados. Já na figura a seguir, reduzimos os dados para metade, ou seja, 50.

<img src="https://i.imgur.com/E9Auwym.png" width="400" height="400" style="float:left">
<img src="https://i.imgur.com/0PQOoTG.png" width="400" height="400" style="float:right">

Finalmente, chega-se ao limite dos dados que consegue processar, ou seja, apenas consegue correr até 5 dados. Abaixo disto, apresenta erros, não mostrando resultados. Na figura a seguir, podemos ver os resultados obtidos, com apenas 5 dados.

<img src="https://i.imgur.com/ObxZyMY.png" width="400" height="400">

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
## TESTES


In [ ]:
#classificadoresBin = ['logistic lasso','logistic ridge','svm','knn']
c_value = [.001, .01, .1, 1, 10]
for i in range(len(c_value)):
    bin_classify(Xtrain2, Xtest2, ytrain, ytest,classificadoresBin[0],cValue[i], solverType = 'liblinear')
    print()

In [ ]:
#classificadoresBin = ['logistic lasso','logistic ridge','svm','knn']
# =============================================================================
# print("-----------------------LOGISTIC lasso - liblinear---------------------------")
# binClassify(x_train, x_test, y1, y2,classificadoresBin[0],cValue = 1, solverType = 'liblinear')
# print()
# print("-----------------------LOGISTIC lasso - saga---------------------------")
# binClassify(x_train, x_test, y1, y2,classificadoresBin[0],cValue = 1, solverType = 'saga')
# print()
# print("-----------------------LOGISTIC ridge - lbfgs---------------------------")
# binClassify(x_train, x_test, y1, y2,classificadoresBin[1], cValue = 10, solverType = 'lbfgs')
# print()
# print("-----------------------LOGISTIC ridge - saga---------------------------")
# binClassify(x_train, x_test, y1, y2,classificadoresBin[1], cValue = 10, solverType = 'saga')
# print()
# print("-----------------------LOGISTIC ridge - sag---------------------------")
# binClassify(x_train, x_test, y1, y2,classificadoresBin[1], cValue = 10, solverType = 'sag')
# print()
# print("-----------------------LOGISTIC ridge - newton-cg---------------------------")
# binClassify(x_train, x_test, y1, y2,classificadoresBin[1], cValue = 10, solverType = 'newton-cg')
# print("-----------------------SVM - linear---------------------------")
# binClassify(x_train, x_test, y1, y2,classificadoresBin[2], cValue = 1, kernelType = 'linear')
# print()
# print("-----------------------SVM - linear---------------------------")
# binClassify(x_train, x_test, y1, y2,classificadoresBin[2], cValue = 1, kernelType = 'linear')
#print()
#print("-----------------------SVM - rbf---------------------------")
#binClassify(x_train, x_test, y1, y2,classificadoresBin[2], cValue = 1, kernelType = 'rbf')
# print()
#print("-----------------------SVM - sigmoid---------------------------")
#binClassify(x_train, x_test, y1, y2,classificadoresBin[2], cValue = 1, kernelType = 'sigmoid')
#print("-----------------------knn---------------------------")
#binClassify(x_train, x_test, y1, y2,classificadoresBin[3], n_neighborsValue = 200)


#classificadoresMulti = ['logistic lasso','logistic ridge','svm','knn']
#----------------------------MULTICLASSE---------------------
#print("-----------------------LOGISTIC lasso - saga---------------------------")
#multiClassify(x_train, x_test, y1, y2,classificadoresMulti[0],cValue = 1, solverType = 'saga')
#print()
#print("-----------------------LOGISTIC ridge - lbfgs---------------------------")
#multiClassify(x_train, x_test, y1, y2,classificadoresMulti[1], cValue = 10, solverType = 'lbfgs')
#print()
#print("-----------------------LOGISTIC ridge - saga---------------------------")
#multiClassify(x_train, x_test, y1, y2,classificadoresMulti[1], cValue = 10, solverType = 'saga')
#print()
#print("-----------------------LOGISTIC ridge - sag---------------------------")
#multiClassify(x_train, x_test, y1, y2,classificadoresMulti[1], cValue = 10, solverType = 'sag')
#print()
#print("-----------------------LOGISTIC ridge - newton-cg---------------------------")
#multiClassify(x_train, x_test, y1, y2,classificadoresMulti[1], cValue = 10, solverType = 'newton-cg')
#print("-----------------------SVM - linear---------------------------")
#multiClassify(x_train, x_test, y1, y2,classificadoresMulti[2], cValue = 1, kernelType = 'linear')
#print("-----------------------SVM - rbf---------------------------")
#multiClassify(x_train, x_test, y1, y2,classificadoresMulti[2], cValue = 1, kernelType = 'rbf')
#print()
#print("-----------------------SVM - sigmoid---------------------------")
#multiClassify(x_train, x_test, y1, y2,classificadoresMulti[2], cValue = 1, kernelType = 'sigmoid')
#print("-----------------------knn---------------------------")
#binClassify(x_train, x_test, y1, y2,classificadoresMulti[3], n_neighborsValue = 200)



![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
## Conclusões<a name="conclusao"/>

Com a realização deste trabalho, conseguimos entender melhor como usufruir dos diferentes classificadores binários e multiclasse que as bibliotecas python fornecem, e obter diferentes resultados consoante os diferentes classificadores utilizados.

O grupo concretizou a maioria dos objetivos propostos no enunciado, o que facilitou a
aprendizagem acerca desta matéria. Inicialmente foram criados métodos para fazer a conversão do texto para vetores, ponto fulcral para o funcionamento dos classificadores utilizados.

Deste modo, os objetivos foram alcançados dentro dos nossos conhecimentos, mas ao longo da realização do trabalho, o grupo deparou-se com problemas, nomeadamente o tempo de processamento que cada classificador consome. Não foi possível realizar todos os testes necessários para obter algumas das comparações dos resultados que pretendia obter, sendo que apenas foram realizados testes para a maioria dos valores aqui apresentados.

Apesar das dificuldades encontradas, foram adquiridos conhecimentos acerca de toda a matéria lecionada, levando mais conhecimentos acerca dos diferentes classificadores, e a função que cada um tem bem como a utilidade da aplicação dos mesmos como métodos de classificação de dados.


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
## Bibliografia<a name="bibliografia"/>

* Folhas fornecidas pelo docente da disciplina;
* Documentação python:
    * Sklearn: https://scikit-learn.org/stable/ ; 
    * Nltk: https://www.nltk.org/ ;
